# Steel Plate Defeat Prediction

In this project I will try to predict, that steel plate will be defeated or not. This is a classification project. I will use feature importances to understand necessary columns. Let's start.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e3/sample_submission.csv
/kaggle/input/playground-series-s4e3/train.csv
/kaggle/input/playground-series-s4e3/test.csv


In [2]:
ss=pd.read_csv("/kaggle/input/playground-series-s4e3/sample_submission.csv")
test=pd.read_csv("/kaggle/input/playground-series-s4e3/test.csv")
train=pd.read_csv("/kaggle/input/playground-series-s4e3/train.csv")

In [3]:
train.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [4]:
test.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,19219,1015,1033,3826564,3826588,659,23,46,62357,67,...,0.0095,0.5652,1.0000,1.0,2.8410,1.1139,1.6628,0.6727,-0.2261,0.9172
1,19220,1257,1271,419960,419973,370,26,28,39293,92,...,0.0047,0.2414,1.0000,1.0,2.5682,0.9031,1.4472,0.9063,-0.1453,0.9104
2,19221,1358,1372,117715,117724,289,36,32,29386,101,...,0.0155,0.6000,0.7500,0.0,2.4609,1.3222,1.3222,-0.5238,-0.0435,0.6514
3,19222,158,168,232415,232440,80,10,11,8586,107,...,0.0037,0.8000,1.0000,1.0,1.9031,0.6990,1.0414,0.1818,-0.0738,0.2051
4,19223,559,592,544375,544389,140,19,15,15524,103,...,0.0158,0.8421,0.5333,0.0,2.1461,1.3222,1.1461,-0.5714,-0.0894,0.4170


In [5]:
train.isnull().sum()

id                       0
X_Minimum                0
X_Maximum                0
Y_Minimum                0
Y_Maximum                0
Pixels_Areas             0
X_Perimeter              0
Y_Perimeter              0
Sum_of_Luminosity        0
Minimum_of_Luminosity    0
Maximum_of_Luminosity    0
Length_of_Conveyer       0
TypeOfSteel_A300         0
TypeOfSteel_A400         0
Steel_Plate_Thickness    0
Edges_Index              0
Empty_Index              0
Square_Index             0
Outside_X_Index          0
Edges_X_Index            0
Edges_Y_Index            0
Outside_Global_Index     0
LogOfAreas               0
Log_X_Index              0
Log_Y_Index              0
Orientation_Index        0
Luminosity_Index         0
SigmoidOfAreas           0
Pastry                   0
Z_Scratch                0
K_Scatch                 0
Stains                   0
Dirtiness                0
Bumps                    0
Other_Faults             0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
y = train[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_multi = MultiOutputClassifier(RandomForestClassifier())
model_multi.fit(X_train, y_train)
y_pred = model_multi.predict(X_test)

for i, target in enumerate(y.columns):
    print(f"{target} Accuracy:", accuracy_score(y_test[target], y_pred[:, i]))


Pastry Accuracy: 0.9227367325702394
Z_Scratch Accuracy: 0.9521331945889698
K_Scatch Accuracy: 0.9646201873048907
Stains Accuracy: 0.9802289281997919
Dirtiness Accuracy: 0.9778876170655567
Bumps Accuracy: 0.7806971904266389
Other_Faults Accuracy: 0.6646722164412071


In [7]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
y = train[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_multi = MultiOutputClassifier(LogisticRegression())
model_multi.fit(X_train, y_train)
y_pred = model_multi.predict(X_test)

for i, target in enumerate(y.columns):
    print(f"{target} Accuracy:", accuracy_score(y_test[target], y_pred[:, i]))


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Pastry Accuracy: 0.9216961498439126
Z_Scratch Accuracy: 0.9427679500520292
K_Scatch Accuracy: 0.9562955254942768
Stains Accuracy: 0.9737252861602498
Dirtiness Accuracy: 0.9778876170655567
Bumps Accuracy: 0.759105098855359
Other_Faults Accuracy: 0.649063475546306


In [8]:
from sklearn.ensemble import RandomForestClassifier

X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y = train[['Stains']]

clf = RandomForestClassifier()
clf.fit(X, y)
pred = clf.predict(X)
feature_importance = clf.feature_importances_
columns=X.columns
feature_importance_sorting = sorted(zip(columns, feature_importance), key=lambda x: x[1], reverse=True)
for f, i in feature_importance_sorting:
    print(f"{f}: {i}")

/tmp/ipykernel_2943/1904085846.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X, y)


Pixels_Areas: 0.1189549922094188
SigmoidOfAreas: 0.11467559868839187
LogOfAreas: 0.11395482171727153
Sum_of_Luminosity: 0.09835116219914154
Y_Perimeter: 0.061808140000622167
Steel_Plate_Thickness: 0.059416758499573896
X_Perimeter: 0.037113646496872314
Length_of_Conveyer: 0.032555679807238065
Luminosity_Index: 0.030795622065751923
Log_Y_Index: 0.029541403575651867
Log_X_Index: 0.027544015781466213
Edges_Index: 0.025180732851956683
X_Minimum: 0.023719524617687462
Empty_Index: 0.02284632404171816
X_Maximum: 0.022683888540080837
Minimum_of_Luminosity: 0.021044319538720862
Y_Maximum: 0.02089167082574326
Y_Minimum: 0.020697912747556675
Edges_X_Index: 0.020397810991557627
Orientation_Index: 0.018331401454764945
Maximum_of_Luminosity: 0.01800733378590315
Outside_X_Index: 0.017281826007338515
Square_Index: 0.0151417660480894
TypeOfSteel_A300: 0.009088406002774553
TypeOfSteel_A400: 0.008005068342513336
Edges_Y_Index: 0.007300052893393312
Outside_Global_Index: 0.004670120268801086


In [9]:

for f, i in feature_importance_sorting:
    if i < 0.025:
        train.drop(f, axis=1, inplace=True)
        test.drop(f, axis=1, inplace=True)
    else:
        continue

In [10]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y = train[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_multi = MultiOutputClassifier(RandomForestClassifier())
model_multi.fit(X_train, y_train)
y_pred = model_multi.predict(X_test)

for i, target in enumerate(y.columns):
    print(f"{target} Accuracy:", accuracy_score(y_test[target], y_pred[:, i]))


Pastry Accuracy: 0.9224765868886576
Z_Scratch Accuracy: 0.9497918834547346
K_Scatch Accuracy: 0.9633194588969823
Stains Accuracy: 0.9820499479708636
Dirtiness Accuracy: 0.9773673257023934
Bumps Accuracy: 0.7757544224765869
Other_Faults Accuracy: 0.6612903225806451


In [11]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler

2024-05-24 11:53:35.308597: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 11:53:35.308678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 11:53:35.310468: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Pastry

In [12]:
X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y=train["Pastry"]
scaler = StandardScaler()
X=scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model=Sequential()
model.add(Dense(128,activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9233 - loss: 0.2803 - val_accuracy: 0.9217 - val_loss: 0.2164
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9230 - loss: 0.2225 - val_accuracy: 0.9217 - val_loss: 0.2119
Epoch 3/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9262 - loss: 0.2085 - val_accuracy: 0.9217 - val_loss: 0.2118
Epoch 4/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9198 - loss: 0.2204 - val_accuracy: 0.9217 - val_loss: 0.2117
Epoch 5/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9266 - loss: 0.2103 - val_accuracy: 0.9217 - val_loss: 0.2131
Epoch 6/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9220 - loss: 0.2169 - val_accuracy: 0.9217 - val_loss: 0.2137
Epoch 7/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9225 - loss: 0.2129 - val_accuracy: 0.9217 - val_loss: 0.2130
Epoch 8/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9242 - loss: 0.2062 - val_accuracy: 0.

In [15]:
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred.round())

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.9216961498439126

In [16]:
model.save("pastry.h5")

# Z_Scratch

In [17]:
X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y=train["Z_Scratch"]
scaler = StandardScaler()
X=scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
model=Sequential()
model.add(Dense(128,activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9325 - loss: 0.2330 - val_accuracy: 0.9428 - val_loss: 0.1467
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9428 - loss: 0.1482 - val_accuracy: 0.9467 - val_loss: 0.1430
Epoch 3/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9406 - loss: 0.1495 - val_accuracy: 0.9428 - val_loss: 0.1301
Epoch 4/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9389 - loss: 0.1442 - val_accuracy: 0.9485 - val_loss: 0.1256
Epoch 5/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9413 - loss: 0.1329 - val_accuracy: 0.9459 - val_loss: 0.1270
Epoch 6/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9423 - loss: 0.1342 - val_accuracy: 0.9461 - val_loss: 0.1245
Epoch 7/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9461 - loss: 0.1264 - val_accuracy: 0.9402 - val_loss: 0.1262
Epoch 8/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9406 - loss: 0.1314 - val_accuracy: 0.

In [20]:
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred.round())

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.946149843912591

In [21]:
model.save("z_scratch.h5")

# K_Scatch

In [22]:
X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y=train["K_Scatch"]
scaler = StandardScaler()
X=scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
model=Sequential()
model.add(Dense(128,activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9258 - loss: 0.2362 - val_accuracy: 0.9586 - val_loss: 0.1257
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9552 - loss: 0.1412 - val_accuracy: 0.9605 - val_loss: 0.1203
Epoch 3/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9559 - loss: 0.1405 - val_accuracy: 0.9610 - val_loss: 0.1200
Epoch 4/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9572 - loss: 0.1358 - val_accuracy: 0.9607 - val_loss: 0.1272
Epoch 5/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9563 - loss: 0.1342 - val_accuracy: 0.9625 - val_loss: 0.1159
Epoch 6/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9582 - loss: 0.1294 - val_accuracy: 0.9625 - val_loss: 0.1189
Epoch 7/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9565 - loss: 0.1355 - val_accuracy: 0.9641 - val_loss: 0.1163
Epoch 8/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9614 - loss: 0.1194 - val_accuracy: 0.

In [25]:
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred.round())

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.9620187304890739

In [26]:
model.save("k_scatch.h5")

# Stains

In [27]:
X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y=train["Stains"]
scaler = StandardScaler()
X=scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
model=Sequential()
model.add(Dense(128,activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9690 - loss: 0.1388 - val_accuracy: 0.9802 - val_loss: 0.0450
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9833 - loss: 0.0461 - val_accuracy: 0.9802 - val_loss: 0.0485
Epoch 3/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9870 - loss: 0.0422 - val_accuracy: 0.9810 - val_loss: 0.0466
Epoch 4/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9858 - loss: 0.0393 - val_accuracy: 0.9810 - val_loss: 0.0436
Epoch 5/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9854 - loss: 0.0425 - val_accuracy: 0.9815 - val_loss: 0.0450
Epoch 6/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9847 - loss: 0.0420 - val_accuracy: 0.9810 - val_loss: 0.0451
Epoch 7/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9857 - loss: 0.0404 - val_accuracy: 0.9820 - val_loss: 0.0459
Epoch 8/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9860 - loss: 0.0397 - val_accuracy: 0.

In [30]:
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred.round())

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.9815296566077003

In [31]:
model.save("stains.h5")

# Dirtiness

In [32]:
X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y=train["Dirtiness"]
scaler = StandardScaler()
X=scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
model=Sequential()
model.add(Dense(128,activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9695 - loss: 0.1621 - val_accuracy: 0.9779 - val_loss: 0.0866
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9727 - loss: 0.1063 - val_accuracy: 0.9779 - val_loss: 0.0829
Epoch 3/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9747 - loss: 0.0960 - val_accuracy: 0.9779 - val_loss: 0.0842
Epoch 4/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9732 - loss: 0.1005 - val_accuracy: 0.9779 - val_loss: 0.0824
Epoch 5/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9779 - loss: 0.0858 - val_accuracy: 0.9781 - val_loss: 0.0848
Epoch 6/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9756 - loss: 0.0900 - val_accuracy: 0.9781 - val_loss: 0.0864
Epoch 7/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9737 - loss: 0.0927 - val_accuracy: 0.9779 - val_loss: 0.0815
Epoch 8/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9714 - loss: 0.1019 - val_accuracy: 0.

In [35]:
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred.round())

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.9765868886576483

In [36]:
model.save("dirtiness.h5")

# Bumps

In [37]:
X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y=train["Bumps"]
scaler = StandardScaler()
X=scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
model=Sequential()
model.add(Dense(128,activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7487 - loss: 0.5044 - val_accuracy: 0.7719 - val_loss: 0.4627
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7549 - loss: 0.4750 - val_accuracy: 0.7680 - val_loss: 0.4585
Epoch 3/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7589 - loss: 0.4716 - val_accuracy: 0.7685 - val_loss: 0.4575
Epoch 4/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7672 - loss: 0.4550 - val_accuracy: 0.7666 - val_loss: 0.4595
Epoch 5/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7664 - loss: 0.4612 - val_accuracy: 0.7747 - val_loss: 0.4565
Epoch 6/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7698 - loss: 0.4554 - val_accuracy: 0.7659 - val_loss: 0.4584
Epoch 7/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7662 - loss: 0.4571 - val_accuracy: 0.7653 - val_loss: 0.4546
Epoch 8/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7626 - loss: 0.4564 - val_accuracy: 0.

In [40]:
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred.round())

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.7661290322580645

In [41]:
model.save("bumps.h5")

# Other Faults

In [42]:
X = train.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults','id'])
y=train["Other_Faults"]
scaler = StandardScaler()
X=scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
model=Sequential()
model.add(Dense(128,activation="relu",input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(16,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6631 - loss: 0.6156 - val_accuracy: 0.6649 - val_loss: 0.5951
Epoch 2/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6703 - loss: 0.5944 - val_accuracy: 0.6644 - val_loss: 0.5949
Epoch 3/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6738 - loss: 0.5942 - val_accuracy: 0.6636 - val_loss: 0.5946
Epoch 4/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6758 - loss: 0.5930 - val_accuracy: 0.6644 - val_loss: 0.5947
Epoch 5/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6858 - loss: 0.5818 - val_accuracy: 0.6655 - val_loss: 0.5958
Epoch 6/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6804 - loss: 0.5832 - val_accuracy: 0.6678 - val_loss: 0.5923
Epoch 7/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6879 - loss: 0.5820 - val_accuracy: 0.6662 - val_loss: 0.5935
Epoch 8/10
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6805 - loss: 0.5833 - val_accuracy: 0.

In [45]:
y_pred=model.predict(x_test)
accuracy_score(y_test,y_pred.round())

121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


0.6664932362122789

In [46]:
model.save("other_faults.h5")

In [47]:
ids=test["id"]
test.drop(columns=["id"],inplace=True)
scaler = StandardScaler()
X=scaler.fit_transform(test)
test=pd.DataFrame(X)

In [51]:
from tensorflow.keras.models import load_model
model_paths=["pastry.h5","z_scratch.h5","k_scatch.h5","stains.h5","dirtiness.h5","bumps.h5","other_faults.h5"]
df=pd.DataFrame()

for path in model_paths:
    model=load_model(path)
    y_pred=model.predict(test)
    y_pred=y_pred.flatten()
    df[path]=y_pred
df["id"]=ids 

401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [52]:
df.head()

,pastry.h5,z_scratch.h5,k_scatch.h5,stains.h5,dirtiness.h5,bumps.h5,other_faults.h5,id
0,0.343620,0.000004,0.011487,1.573974e-11,0.018955,0.192946,0.471887,19219
1,0.206978,0.012543,0.019573,2.298840e-08,0.077382,0.182735,0.418967,19220
2,0.016399,0.057544,0.130667,1.173492e-05,0.010478,0.299034,0.453400,19221
3,0.216903,0.000037,0.000068,2.246642e-03,0.008015,0.269647,0.399083,19222
4,0.005497,0.001310,0.000501,8.460208e-05,0.007703,0.606711,0.375831,19223


In [53]:
ss.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,19220,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,19221,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,19222,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,19223,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [54]:
df.rename({"pastry.h5":"Pastry","z_scratch.h5":"Z_Scratch","k_scatch.h5":"K_Scatch","stains.h5":"Stains","dirtiness.h5":"Dirtiness","bumps.h5":"Bumps","other_faults.h5":"Other_Faults"},axis=1,inplace=True)
df.head()

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,id
0,0.343620,0.000004,0.011487,1.573974e-11,0.018955,0.192946,0.471887,19219
1,0.206978,0.012543,0.019573,2.298840e-08,0.077382,0.182735,0.418967,19220
2,0.016399,0.057544,0.130667,1.173492e-05,0.010478,0.299034,0.453400,19221
3,0.216903,0.000037,0.000068,2.246642e-03,0.008015,0.269647,0.399083,19222
4,0.005497,0.001310,0.000501,8.460208e-05,0.007703,0.606711,0.375831,19223


In [55]:
df.to_csv("submission.csv",index=False)

In this project I tried to predict, that steel plate will be defeated or not. Some metrics ar not satisfactory. But metrics are not enough alone to understand performance of model. That's why I'll write a Streamlit app per a model.